https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.datasets.QM9.html

In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))

import torch
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from gtda.plotting import plot_diagram

from src.dataset import load_qm9
from src.tda import TDAEngine

%matplotlib inline

In [2]:
DATA_ROOT = '../data'
dataset = load_qm9(root=DATA_ROOT)

In [3]:
print("Молекул:", len(dataset))

Молекул: 130831


Выберем молекулу с индексом, например, `100`.

In [4]:
molecule_idx = 100
sample_molecule = dataset[molecule_idx]

In [5]:
type(sample_molecule)

torch_geometric.data.data.Data

Атрибуты молекулы

In [6]:
sample_molecule

Data(x=[11, 11], edge_index=[2, 22], edge_attr=[22, 4], y=[1, 19], pos=[11, 3], idx=[1], name='gdb_104', z=[11])

Порядковый номер молекулы в оригинальном датасете

In [7]:
sample_molecule.idx

tensor([103])

Уникальное название молекулы в БД GDB-17

In [8]:
sample_molecule.name

'gdb_104'

Номера атомов

In [9]:
sample_molecule.z

tensor([8, 6, 6, 6, 8, 1, 1, 1, 1, 1, 1])

3D-координаты атомов

In [10]:
sample_molecule.pos

tensor([[-0.2170,  1.3823, -0.0107],
        [-0.0426, -0.0106,  0.0036],
        [-0.4454, -0.7881,  1.2394],
        [-1.3388, -0.7701,  0.0145],
        [-2.5191, -0.0351,  0.0349],
        [ 0.3113,  1.7667,  0.6959],
        [ 0.7915, -0.3680, -0.5970],
        [ 0.1076, -1.6883,  1.4859],
        [-0.7973, -0.2116,  2.0892],
        [-1.4224, -1.6756, -0.5787],
        [-2.2517,  0.8938, -0.0114]])

Связи (рёбра графа) в формате пары индексов: начала и конца связи

In [11]:
sample_molecule.edge_index

tensor([[ 0,  0,  1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  4,  4,  5,  6,
          7,  8,  9, 10],
        [ 1,  5,  0,  2,  3,  6,  1,  3,  7,  8,  1,  2,  4,  9,  3, 10,  0,  1,
          2,  2,  3,  4]])

Признаки связей:

- [1, 0, 0, 0] — одинарная связь
- [0, 1, 0, 0] — двойная связь
- [0, 0, 1, 0] — тройная связь
- [0, 0, 0, 1] — ароматическая связь

In [12]:
sample_molecule.edge_attr

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]])

Целевые переменные:

| Target | Свойство                     | Описание                                              | Единица         |
|--------|------------------------------|-------------------------------------------------------|-----------------|
| 0      | $\mu$                        | Дипольный момент                                      | $\mathrm{D}$    |
| 1      | $\alpha$                     | Изотропная поляризуемость                             | $a_0^3$         |
| 2      | $\varepsilon_{\text{HOMO}}$  | Наивысшая занятая молекулярная орбитальная энергия   | $\mathrm{eV}$   |
| 3      | $\varepsilon_{\text{LUMO}}$  | Низшая незанятая молекулярная орбитальная энергия    | $\mathrm{eV}$   |
| 4      | $\Delta\varepsilon$          | Разность между $\varepsilon_{\text{HOMO}}$ и $\varepsilon_{\text{LUMO}}$ | $\mathrm{eV}$   |
| 5      | $\langle R^2 \rangle$        | Электронная пространственная протяжённость           | $a_0^2$         |
| 6      | $\text{ZPVE}$                | Энергия нулевых колебаний                             | $\mathrm{eV}$   |
| 7      | $U_0$                        | Внутренняя энергия при 0 K                            | $\mathrm{eV}$   |
| 8      | $U$                          | Внутренняя энергия при 298.15 K                       | $\mathrm{eV}$   |
| 9      | $H$                          | Энтальпия при 298.15 K                                | $\mathrm{eV}$   |
| 10     | $G$                          | Свободная энергия Гиббса при 298.15 K                 | $\mathrm{eV}$   |
| 11     | $c_v$                        | Теплоёмкость при постоянном объёме при 298.15 K       | $\frac{\mathrm{cal}}{\mathrm{mol\cdot K}}$ |
| 12     | $U_0^{\text{ATOM}}$          | Атомизация энергии при 0 K                            | $\mathrm{eV}$   |
| 13     | $U^{\text{ATOM}}$            | Атомизация энергии при 298.15 K                       | $\mathrm{eV}$   |
| 14     | $H^{\text{ATOM}}$            | Энтальпия атомизации при 298.15 K                     | $\mathrm{eV}$   |
| 15     | $G^{\text{ATOM}}$            | Свободная энергия атомизации при 298.15 K            | $\mathrm{eV}$   |
| 16     | $A$                          | Вращательная постоянная                               | $\mathrm{GHz}$  |
| 17     | $B$                          | Вращательная постоянная                               | $\mathrm{GHz}$  |
| 18     | $C$                          | Вращательная постоянная                               | $\mathrm{GHz}$  |

In [13]:
sample_molecule.y

tensor([[ 2.4925e+00,  3.8430e+01, -6.4464e+00,  1.7470e+00,  8.1933e+00,
          3.7028e+02,  2.4598e+00, -7.2995e+03, -7.2993e+03, -7.2993e+03,
         -7.3002e+03,  1.9173e+01, -4.2994e+01, -4.3275e+01, -4.3532e+01,
         -4.0010e+01,  7.5116e+00,  5.3853e+00,  3.8031e+00]])

Визуализация молекулы

In [18]:
pos = sample_molecule.pos.numpy()
z = sample_molecule.z.numpy()

elements = {1: 'H', 6: 'C', 7: 'N', 8: 'O'}
colors = {1: 'blue', 6: 'black', 7: 'lightgray', 8: 'white'}

atom_trace = go.Scatter3d(
    x=pos[:, 0], y=pos[:, 1], z=pos[:, 2],
    mode='markers',
    marker=dict(
        size=10,
        color=[colors.get(atom, 'yellow') for atom in z],
        line=dict(color='black', width=2)
    ),
    text=[f"Атом: {elements.get(atom, atom)} (idx: {i})" for i, atom in enumerate(z)],
    hoverinfo='text',
    name='Атомы'
)

edge_x, edge_y, edge_z = [], [], []

edges = sample_molecule.edge_index.numpy()

for i in range(edges.shape[1]):
    start_idx = edges[0, i]
    end_idx = edges[1, i]
    edge_x.extend([pos[start_idx, 0], pos[end_idx, 0], None])
    edge_y.extend([pos[start_idx, 1], pos[end_idx, 1], None])
    edge_z.extend([pos[start_idx, 2], pos[end_idx, 2], None])

bond_trace = go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode='lines',
    line=dict(color='gray', width=5),
    hoverinfo='none',
    name='Связи'
)

layout = go.Layout(
    title=f'Структура молекулы {molecule_idx}',
    scene=dict(
        xaxis=dict(title='X (Å)', backgroundcolor="rgb(200, 200, 230)"),
        yaxis=dict(title='Y (Å)', backgroundcolor="rgb(230, 200, 230)"),
        zaxis=dict(title='Z (Å)', backgroundcolor="rgb(230, 230, 200)"),
        aspectmode='data'
    ),
    margin=dict(l=0, r=0, b=0, t=40),
    showlegend=True,
    height=720,
    width=1280
)

fig = go.Figure(data=[bond_trace, atom_trace], layout=layout)
fig.show()